<a href="https://colab.research.google.com/github/Shubhanshu1999/Dev-Con/blob/master/GAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import keras
from keras.preprocessing import image
from keras.engine import Layer
from keras.layers import Conv2D, Conv3D, UpSampling2D, InputLayer, Conv2DTranspose, Input, Reshape, merge, Concatenate, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Activation, Dense, Dropout, Flatten, Lambda
from keras.layers.normalization import BatchNormalization
from keras.callbacks import TensorBoard
from keras.models import Sequential, Model
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from skimage.color import rgb2lab, lab2rgb, rgb2gray, gray2rgb
from skimage.transform import resize
from skimage.io import imsave
from time import time
import numpy as np
import os
import random
from PIL import Image, ImageFile

In [ ]:
import scipy.io as scio
mat=scio.loadmat('/content/drive/MyDrive/final_year_project/folder/data0.mat')
X_train=mat['Data_input']
Y_train=mat['Data_output']
for i in range(1,10):
  mat=scio.loadmat('/content/drive/MyDrive/final_year_project/folder/data'+str(i)+'.mat')
  X_train=np.concatenate((X_train,mat['Data_input']),axis=0)
  Y_train=np.concatenate((Y_train,mat['Data_output']),axis=0)
X_train=X_train.astype('float32')/100
Y_train=Y_train.astype('float32')/100
X_train=np.expand_dims(X_train,axis=-1)

mat=scio.loadmat('/content/drive/MyDrive/final_year_project/folder/data10.mat')
X_test=mat['Data_input']
Y_test=mat['Data_output']
for i in range(11,13):
  mat=scio.loadmat('/content/drive/MyDrive/final_year_project/folder/data'+str(i)+'.mat')
  X_test=np.concatenate((X_test,mat['Data_input']),axis=0)
  Y_test=np.concatenate((Y_test,mat['Data_output']),axis=0)
X_test=X_test.astype('float32')/100
Y_test=Y_test.astype('float32')/100
X_test=np.expand_dims(X_test,axis=-1)


In [ ]:
#Encoder
encoder_input = Input(shape=(224,224,1))
vggmodel = keras.applications.vgg16.VGG16()
newmodel = Sequential() 
#newmodel.add(Input(shape=(224,224,1)))
#newmodel.add(Concatenate(axis=-1)([encoder_input,encoder_input,encoder_input]))
num = 0
for i, layer in enumerate(vggmodel.layers):
    if i<19:
      newmodel.add(layer)
for layer in newmodel.layers:
  layer.trainable=False

x = Concatenate(axis=-1)([encoder_input,encoder_input,encoder_input])
model_tmp = Model(inputs=encoder_input, outputs=x)
model_tmp.compile(optimizer='Adam', loss='mse')
model_encoder=Model(inputs=encoder_input,outputs=newmodel(model_tmp(encoder_input)))
model_encoder.compile(optimizer='Adam', loss='mse')
model_encoder.summary()


#Decoder
decoder_input = Input(shape=(7,7,512))
decoder_output = Conv2D(256, (3,3), activation='relu', padding='same')(decoder_input)
decoder_output = Conv2D(128, (3,3), activation='relu', padding='same')(decoder_output)
decoder_output = UpSampling2D((2, 2))(decoder_output)
decoder_output = Conv2D(64, (3,3), activation='relu', padding='same')(decoder_output)
decoder_output = UpSampling2D((2, 2))(decoder_output)
decoder_output = Conv2D(32, (3,3), activation='relu', padding='same')(decoder_output)
decoder_output = UpSampling2D((2, 2))(decoder_output)
decoder_output = Conv2D(16, (3,3), activation='relu', padding='same')(decoder_output)
decoder_output = UpSampling2D((2, 2))(decoder_output)
decoder_output = Conv2D(2, (3, 3), activation='tanh', padding='same')(decoder_output)
decoder_output = UpSampling2D((2, 2))(decoder_output)
decoder_output = Concatenate(axis=-1)([encoder_input,decoder_output])
model_decoder=Model(inputs=[encoder_input,decoder_input],outputs=decoder_output)
model_decoder.compile(optimizer='Adam', loss='mse')
model_decoder.summary()
model_gen = Model(inputs=encoder_input, outputs=model_decoder([encoder_input,model_encoder(encoder_input)]))
model_gen.summary()

Model: "model_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         [(None, 224, 224, 1)]     0         
_________________________________________________________________
model_8 (Functional)         (None, 224, 224, 3)       0         
_________________________________________________________________
sequential_2 (Sequential)    (None, 7, 7, 512)         14714688  
Total params: 14,714,688
Trainable params: 0
Non-trainable params: 14,714,688
_________________________________________________________________
Model: "model_10"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_9 (InputLayer)            [(None, 7, 7, 512)]  0                                            
______________________________________________________________

In [ ]:
model_encoder.summary()

Model: "model_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         [(None, 224, 224, 1)]     0         
_________________________________________________________________
model_8 (Functional)         (None, 224, 224, 3)       0         
_________________________________________________________________
sequential_2 (Sequential)    (None, 7, 7, 512)         14714688  
Total params: 14,714,688
Trainable params: 0
Non-trainable params: 14,714,688
_________________________________________________________________


In [ ]:
model_decoder.summary()

Model: "model_10"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_9 (InputLayer)            [(None, 7, 7, 512)]  0                                            
__________________________________________________________________________________________________
conv2d_12 (Conv2D)              (None, 7, 7, 256)    1179904     input_9[0][0]                    
__________________________________________________________________________________________________
conv2d_13 (Conv2D)              (None, 7, 7, 128)    295040      conv2d_12[0][0]                  
__________________________________________________________________________________________________
up_sampling2d_10 (UpSampling2D) (None, 14, 14, 128)  0           conv2d_13[0][0]                  
___________________________________________________________________________________________

In [ ]:
model_encoder.summary()

Model: "model_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         [(None, 224, 224, 1)]     0         
_________________________________________________________________
model_8 (Functional)         (None, 224, 224, 3)       0         
_________________________________________________________________
sequential_2 (Sequential)    (None, 7, 7, 512)         14714688  
Total params: 14,714,688
Trainable params: 0
Non-trainable params: 14,714,688
_________________________________________________________________


In [ ]:
from keras.callbacks import TensorBoard
tensorboard = TensorBoard(log_dir='/content/logs')
model_gen.compile(optimizer='Adam', loss='mse' , metrics=['accuracy'])
critic_input = Input(shape=(224, 224, 3,))
x = Conv2D(64, (3,3), activation='relu')(critic_input)
x = Conv2D(64, (3,3), activation='relu')(x)
x = Conv2D(64, (3,3), activation='relu')(x)
x = MaxPooling2D((2,2))(x)
x = Conv2D(128, (3,3), activation='relu')(x)
x = Conv2D(128, (3,3), activation='relu')(x)
x = Conv2D(128, (3,3), activation='relu')(x)
x = MaxPooling2D((2,2))(x)
x = Conv2D(256, (3,3), activation='relu')(x)
x = Conv2D(256, (3,3), activation='relu')(x)
x = Conv2D(256, (3,3), activation='relu')(x)
x = MaxPooling2D((2,2))(x)
x = Conv2D(512, (3,3), activation='relu')(x)
x = Conv2D(512, (3,3), activation='relu')(x)
x = Conv2D(512, (3,3), activation='relu')(x)
x = MaxPooling2D((2,2))(x)
x = GlobalAveragePooling2D()(x)
x = Dense(1,activation='softmax')(x)
critic_model=Model(inputs=critic_input, outputs=x)
critic_model.compile(optimizer='Adam', loss='binary_crossentropy' , metrics=['accuracy'])

In [ ]:
critic_model.summary()

Model: "model_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_10 (InputLayer)        [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 222, 222, 64)      1792      
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 220, 220, 64)      36928     
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 218, 218, 64)      36928     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 109, 109, 64)      0         
_________________________________________________________________
conv2d_21 (Conv2D)           (None, 107, 107, 128)     73856     
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 105, 105, 128)     147

In [ ]:
gan_model = Model(inputs=encoder_input,outputs=[model_gen(encoder_input),critic_model(model_gen(encoder_input))])
gan_model.compile(optimizer='Adam', loss=['mse','binary_crossentropy'],loss_weights=[.9,.1])

In [ ]:
gan_model.layers[1].summary()


Model: "model_11"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            [(None, 224, 224, 1) 0                                            
__________________________________________________________________________________________________
model_9 (Functional)            (None, 7, 7, 512)    14714688    input_7[0][0]                    
__________________________________________________________________________________________________
model_10 (Functional)           (None, 224, 224, 3)  1572114     input_7[0][0]                    
                                                                 model_9[0][0]                    
Total params: 16,286,802
Trainable params: 1,572,114
Non-trainable params: 14,714,688
__________________________________________________________________________________________________


In [ ]:
train_positive = np.ones((X_train.shape[0],1),dtype=np.float32)
train_negative = np.zeros((X_train.shape[0],1),dtype=np.float32)
#train_negative = -train_positive

test_positive = np.ones((X_test.shape[0],1),dtype=np.float32)
test_negative = np.zeros((X_test.shape[0],1),dtype=np.float32)
#test_negative =-test_positive


In [ ]:
train_positive.shape

(750, 1)

In [ ]:
for iter in range(0,15):
  gn_op_train=gan_model.layers[1].predict(X_train)
  gn_op_test=gan_model.layers[1].predict(X_test)
  gan_model.layers[1].trainable=False
  gan_model.layers[2].trainable=True
  gan_model.layers[2].compile(loss='binary_crossentropy', metrics='accuracy', optimizer='Adam')
  gan_model.layers[2].fit(np.concatenate((Y_train,gn_op_train),axis=0),np.concatenate((train_positive,train_negative),axis=0),
                          validation_data=(np.concatenate((Y_test,gn_op_test),axis=0),np.concatenate((test_positive,test_negative),axis=0)),
                          batch_size=16,epochs=5,shuffle=True)
  gan_model.layers[1].trainable=True
  gan_model.layers[2].trainable=False
  gan_model.compile(loss=['mse','binary_crossentropy'],loss_weights=[.9,.1], metrics='accuracy', optimizer='Adam')
  gan_model.fit(X_train,[Y_train,train_positive],validation_data=(X_test,[Y_test,test_positive]),batch_size=16,epochs=3,shuffle=True)

Epoch 1/5
94/94 [==============================] - 36s 301ms/step - loss: 0.6255 - accuracy: 0.5000 - val_loss: 0.0825 - val_accuracy: 0.5000
Epoch 2/5
94/94 [==============================] - 20s 218ms/step - loss: 0.0728 - accuracy: 0.4679 - val_loss: 0.0407 - val_accuracy: 0.5000
Epoch 3/5
94/94 [==============================] - 20s 218ms/step - loss: 2.7332 - accuracy: 0.5062 - val_loss: 0.0826 - val_accuracy: 0.5000
Epoch 4/5
94/94 [==============================] - 20s 217ms/step - loss: 29.2513 - accuracy: 0.4661 - val_loss: 0.6035 - val_accuracy: 0.5000
Epoch 5/5
94/94 [==============================] - 21s 219ms/step - loss: 258.9368 - accuracy: 0.4767 - val_loss: 0.6936 - val_accuracy: 0.5000
Epoch 1/3
47/47 [==============================] - 48s 808ms/step - loss: 0.1259 - model_11_loss: 0.0647 - model_12_loss: 0.6767 - model_11_accuracy: 0.8112 - model_12_accuracy: 1.0000 - val_loss: 0.0816 - val_model_11_loss: 0.0142 - val_model_12_loss: 0.6876 - val_model_11_accuracy: 0.

In [ ]:
gan_model.layers[0]

In [ ]:
gan_model.output

[<KerasTensor: shape=(None, 224, 224, 3) dtype=float32 (created by layer 'model_3')>,
 <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'model_4')>]

In [ ]:
lab=gan_model.layers[1].predict(X_test)

In [ ]:
for i in range(0,225):
    #print(L.shape)
    img=lab[i]
    img=img*100
    img=lab2rgb(img)
    img=img*255
    print(img.shape)
    imsave('/content/results/'+str(i)+".jpg", img.astype(np.uint8))

(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 

In [ ]:
gan_model.save('/content/drive/MyDrive/final_year_project/saved_models') 

INFO:tensorflow:Assets written to: /content/drive/MyDrive/final_year_project/saved_models/assets


In [ ]:
!zip -r /content/results.zip /content/results

In [ ]:
import shutil

shutil.rmtree('/content/results')